In [ ]:

import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

# Read stage files
df = session.read.json('@"SNOWFLAKE_CONNECTED_GCS_US"."PUBSUB_EVENTS"."MY_GCS_STAGE"/')